In [10]:
from qiskit.algorithms.minimum_eigensolvers import QAOA                             
from qiskit.algorithms.optimizers import COBYLA, GradientDescent
from qiskit_aer.primitives import Sampler
from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit_finance.data_providers import RandomDataProvider
from qiskit_optimization.algorithms import MinimumEigenOptimizer
import datetime

$$
\begin{aligned}
\min_{x \in \{0, 1\}^n}\quad& q x^T \Sigma x - \mu^T x\\
\text{s.t.}\quad& 1^T x = B
\end{aligned}
$$

定义基本的参数，构造数据。

In [2]:
num_assets = 4
data = RandomDataProvider(
    tickers=[f"TICKERS{i}" for i in range(num_assets)],
    start=datetime.datetime(2021, 11, 17),
    end=datetime.datetime(2022, 11, 17),
)
data.run()
mu = data.get_period_return_mean_vector()
sigma = data.get_period_return_covariance_matrix()
q = 0.5
B = num_assets // 2

定义投资组合优化问题。

In [3]:
problem = PortfolioOptimization(mu, sigma, q, B)

将原问题转化为一个二次规划问题。

In [4]:
quadratic_program = problem.to_quadratic_program()
print(quadratic_program)

minimize 8.085708164407747e-05*x_0^2 - 1.7716036931802456e-05*x_0*x_1 + 1.799081577323901e-05*x_0*x_2 - 4.4718940017546775e-05*x_0*x_3 + 0.027826327522665822*x_1^2 + 0.0005656342072605836*x_1*x_2 + 0.0004270364614573808*x_1*x_3 + 0.0003801641412271594*x_2^2 - 0.00015515012991674287*x_2*x_3 + 0.02534512718521647*x_3^2 - 0.0005512569075137799*x_0 - 0.024404347420446462*x_1 - 0.001435091910419631*x_2 - 0.009455386343100539*x_3 (4 variables, 1 constraints, 'Portfolio optimization')


In [5]:
problem

In [19]:
quadratic_program.variables


[<Variable: x_0 (binary)>,
 <Variable: x_1 (binary)>,
 <Variable: x_2 (binary)>,
 <Variable: x_3 (binary)>]

利用QAOA求解，得到投资方案。

In [7]:
result = MinimumEigenOptimizer(QAOA(Sampler(), COBYLA(),reps=3)).solve(
    quadratic_program
)
print(result)

fval=-0.001507336779288935, x_0=1.0, x_1=0.0, x_2=1.0, x_3=0.0, status=SUCCESS


使用梯度优化器

In [11]:
result = MinimumEigenOptimizer(QAOA(Sampler(), GradientDescent(),reps=3)).solve(
    quadratic_program
)
print(result)

fval=-0.001507336779288935, x_0=1.0, x_1=0.0, x_2=1.0, x_3=0.0, status=SUCCESS


输出全部的可能方案，得到满足约束方案的概率大。

In [8]:
print("投资方案\t\t概率\t目标函数值")
for (
    k,
    v,
) in sorted(
    result.min_eigen_solver_result.eigenstate.binary_probabilities().items(),
    key=lambda x: x[1],
    reverse=True,
):
    x = [int(i) for i in list(reversed(k))]
    print(
        f"{x}\t{v:.4f}\t{problem.to_quadratic_program().objective.evaluate(x):.6f}"
    )

投资方案		概率	目标函数值
[0, 1, 1, 0]	0.1826	0.002933
[1, 0, 0, 1]	0.1738	0.015375
[1, 1, 0, 0]	0.1611	0.002934
[0, 0, 1, 1]	0.1602	0.014680
[1, 0, 1, 0]	0.1523	-0.001507
[0, 1, 0, 1]	0.1475	0.019739
[1, 0, 0, 0]	0.0059	-0.000470
[0, 0, 1, 0]	0.0029	-0.001055
[1, 1, 1, 1]	0.0029	0.018579
[0, 0, 0, 0]	0.0020	0.000000
[1, 1, 1, 0]	0.0020	0.002463
[1, 1, 0, 1]	0.0020	0.019206
[0, 0, 0, 1]	0.0020	0.015890
[1, 0, 1, 1]	0.0010	0.014183
[0, 1, 0, 0]	0.0010	0.003422
[0, 1, 1, 1]	0.0010	0.019094


In [33]:
unconstraint_quadratic_problem = problem.to_quadratic_program()
unconstraint_quadratic_problem.remove_linear_constraint(0)

result = MinimumEigenOptimizer(QAOA(Sampler(), COBYLA(),reps=3)).solve(
    quadratic_program
)
print(result)

fval=-0.001507336779288935, x_0=1.0, x_1=0.0, x_2=1.0, x_3=0.0, status=SUCCESS


## Continuous Var

In [49]:
continuous_var_quadratic_problem = problem.to_quadratic_program()

In [51]:
continuous_var_quadratic_problem._variables.clear()
continuous_var_quadratic_problem._variables_index.clear()
for i in range(4):
    name = "x"+str(i)
    continuous_var_quadratic_problem.continuous_var(0,B,name=name)

In [52]:
continuous_var_quadratic_problem

<QuadraticProgram: minimize 8.085708164407747e-05*x0^2 - 1.7716036931802456e-0..., 4 variables, 1 constraints, 'Portfolio optimization'>

In [53]:
continuous_var_quadratic_problem.variables

[<Variable: 0 <= x0 <= 2 (continuous)>,
 <Variable: 0 <= x1 <= 2 (continuous)>,
 <Variable: 0 <= x2 <= 2 (continuous)>,
 <Variable: 0 <= x3 <= 2 (continuous)>]